In [83]:
from utils import get_grouped_df

# df = get_grouped_df()
# df.head()

In [84]:
# Charger les données
df = get_grouped_df()
df.head()

/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,nom_acteur,nom_compagnie,nom_genre
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,"John Leguizamo, Jon Favreau, Sofía Vergara, Em...","Altus Media, Fairview Entertainment, Prescience",Comédie
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,"Scarlett Johansson, Jillian Bell, Ilana Glazer...","Paulilu Productions, Sony Pictures, Columbia P...","Drame, Comédie"
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,"Zooey Deschanel, Joseph Gordon-Levitt, Chloë G...","Fox Searchlight Pictures, Watermark Pictures, ...","Comédie, Drame, Romance"
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,"Nanou Garcia, Gilles Lellouche, Valérie Lemerc...","SCOPE Pictures, Rectangle Productions",Comédie
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,"Cliff Curtis, Steven Strait, Nathanael Baring,...","Centropolis Entertainment, Legendary Pictures,...","Aventure, Action, Drame, Fantastique"


In [85]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from azureml.core import Workspace, Experiment, Run
from utils import get_grouped_df
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Charger les données
df = get_grouped_df()

# Assurez-vous d'avoir le titre du film
df_title = df['title'].copy()

# Séparer les listes d'acteurs, de compagnies, et de genres en plusieurs colonnes
df['nom_acteur'] = df['nom_acteur'].str.split(', ')
df['nom_compagnie'] = df['nom_compagnie'].str.split(', ')
df['nom_genre'] = df['nom_genre'].str.split(', ')

# Créer un binarizer
mlb = MultiLabelBinarizer()

# Transformer les colonnes
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_acteur')),
                          columns=[f'actor_{cls}' for cls in mlb.classes_],
                          index=df.index))

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_compagnie')),
                          columns=[f'company_{cls}' for cls in mlb.classes_],
                          index=df.index))

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_genre')),
                          columns=[f'genre_{cls}' for cls in mlb.classes_],
                          index=df.index))

# On remplit les valeurs manquantes par 0
df = df.fillna(0)

df

/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,genre_Fantastique,genre_Guerre,genre_Histoire,genre_Horreur,genre_Musique,genre_Mystère,genre_Romance,genre_Science-Fiction,genre_Thriller,genre_Western
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# Import des librairies
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# forest
from sklearn.ensemble import RandomForestRegressor


# One-hot encoding pour la colonne 'realisateur'
# One-hot encoding pour la colonne 'realisateur'
encoder = OneHotEncoder(sparse=False)
realisateur_encoded = encoder.fit_transform(df[['realisateur']])
# Obtenez les noms des fonctionnalités
feature_names = encoder.categories_[0]
# Créez le DataFrame avec les noms des fonctionnalités
df_encoded = pd.DataFrame(realisateur_encoded, columns=feature_names)


# Concaténez les dataframes
df = pd.concat([df, df_encoded], axis=1)


/home/apprenant/miniconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [87]:
df

,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,Zal Batmanglij,Zhang Yimou,Zhao Ji,Ziad Doueiri,Álex de la Iglesia,Élie Semoun,Émile Gaudreault,Éric Besnard,Éric Toledano,Étienne Chatiliez
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
df

,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,Zal Batmanglij,Zhang Yimou,Zhao Ji,Ziad Doueiri,Álex de la Iglesia,Élie Semoun,Émile Gaudreault,Éric Besnard,Éric Toledano,Étienne Chatiliez
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# Supprimer les colonnes non numériques
df = df.drop(['title', 'date', 'realisateur', 'titre_non_modifie', 'film_id'], axis=1)

# Sélection des features et de la cible
features = df.drop('entries', axis=1)
target = df['entries']

# Division des données en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.22, random_state=42)

# Création de l'objet PolynomialFeatures
poly = PolynomialFeatures(degree=2)

# Définition des transformations : "budget" subit une transformation polynomiale
# les autres variables restent inchangées
transformer = ColumnTransformer(
    transformers=[
        ('poly', poly, ['budget'])],
    remainder='passthrough')

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Création de pipelines pour chaque modèle
pipeline_lr = Pipeline(steps=[('t', transformer),
                           ('model', LinearRegression())])

pipeline_rf = Pipeline(steps=[('t', transformer),
                           ('model', RandomForestRegressor(random_state=42))])

pipeline_gb = Pipeline(steps=[('t', transformer),
                           ('model', GradientBoostingRegressor(random_state=42))])

pipeline_svr = Pipeline(steps=[('t', transformer),
                           ('model', SVR())])

# Paramètres pour GridSearchCV pour chaque modèle
param_grid_lr = {}
param_grid_rf = {'model__n_estimators': [50, 100, 200], 'model__max_depth': [None, 5, 10, 20]}
param_grid_gb = {'model__n_estimators': [50, 100, 200], 'model__learning_rate': [0.01, 0.1, 1.0]}
param_grid_svr = {'model__C': [0.1, 1.0, 10.0], 'model__kernel': ['linear', 'poly', 'rbf']}

# Création d'une liste de modèles à essayer
models = {'lr': (pipeline_lr, param_grid_lr), 
          'rf': (pipeline_rf, param_grid_rf),
          'gb': (pipeline_gb, param_grid_gb),
          'svr': (pipeline_svr, param_grid_svr)}

# Loop sur chaque modèle et faire un GridSearchCV
for name, (model, params) in models.items():
    grid_search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print(f"Model: {name}")
    print("Best Parameters: ", grid_search.best_params_)
    print("\n")
    
    
model = grid_search.best_estimator_

Model: lr
Best Parameters:  {}




KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prédictions du modèle
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

# Convertir les prédictions en entier
train_preds = train_preds.astype(int)
test_preds = test_preds.astype(int)

# Calcul des métriques
train_rmse = sqrt(mean_squared_error(y_train, train_preds))
test_rmse = sqrt(mean_squared_error(y_test, test_preds))

train_mae = mean_absolute_error(y_train, train_preds)
test_mae = mean_absolute_error(y_test, test_preds)

train_r2 = r2_score(y_train, train_preds)
test_r2 = r2_score(y_test, test_preds)

# Affichage des métriques
print(f"Train RMSE: {train_rmse}, Test RMSE: {test_rmse}")
print(f"Train MAE: {train_mae}, Test MAE: {test_mae}")
print(f"Train R2: {train_r2}, Test R2: {test_r2}")

Train RMSE: 210730.36861679188, Test RMSE: 278426.49836908677
Train MAE: 145311.93694535265, Test MAE: 167485.13553719007
Train R2: 0.7826299552179684, Test R2: 0.5446864472134034


In [ ]:
# On crée un DataFrame pour les prédictions de test
df_test = pd.DataFrame({'Title': df_title.loc[X_test.index], 'Actual': y_test, 'Predicted': test_preds})

# voir le actual le plus haut
df_test.sort_values(by='Actual', ascending=False).head(30)

,Title,Actual,Predicted
841,harrypotteretlacoupedefeu,3088796,2272693
1480,leseigneurdesanneauxleretourduroi,2935984,1540325
212,avatar,2925087,1321502
840,harrypotteretlachambredessecrets,2728766,1304893
2018,piratesdescaraibesjusquauboutdumonde,2709377,1606069
2234,shrek2,2238286,950746
1133,lagedeglace2,2171810,435960
2366,taxi4,2010736,694702
218,avengersleredultron,1926049,2779804
1501,lesindestructibles2,1840492,800059
